https://github.com/changhoonhahn/SEDflow/blob/4561ecfe3a38cc4c25df263d971a87e8a83f88ce/bin/train/anpe.py#L41

In [1]:
import os, sys
import numpy as np
import pickle 
import sys
sys.path.append('/media/SSD/Doktori/Csillagtomegbecsles_cikk/SED/SEDflow/SEDflow/src/')
from sedflow import train as Train
# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from sbi import utils as Ut
from sbi import inference as Inference

In [2]:
sample = 'toy'
itrain = 2
nhidden = 500
nblocks = 15
print('%s training data; model %i' % (sample, itrain))

if torch.cuda.is_available(): device = 'cuda'
else: device = 'cpu'

toy training data; model 2


In [3]:
# load training data (theta_sps, mag, sig, z) 
x_train, y_train = Train.load_data('train', version=1, sample=sample, params='thetas_unt') 
# convert gamma1, gamma2 (ZH NMF coefficients) to log space 
x_train[:,6] = np.log10(x_train[:,6])
x_train[:,7] = np.log10(x_train[:,7])
print('Ntrain = %i' % x_train.shape[0])
print('%i dim theta; %i dim obs' % (x_train.shape[1], y_train.shape[1]))

Ntrain = 551413
11 dim theta; 11 dim obs


In [4]:
# prior 
prior_low   = [7, 0., 0., 0., 0., 1e-2, np.log10(4.5e-5), np.log10(4.5e-5), 0, 0., -2.]
prior_high  = [12.5, 1., 1., 1., 1., 13.27, np.log10(1.5e-2), np.log10(1.5e-2), 3., 3., 1.]
lower_bounds = torch.tensor(prior_low).to(device)
upper_bounds = torch.tensor(prior_high).to(device)
prior = Ut.BoxUniform(low=lower_bounds, high=upper_bounds, device=device)

In [5]:
# train NPE  
fanpe = os.path.join(Train.data_dir(), 'anpe_thetaunt_magsigz.%s.%ix%i.%i.pt' % (sample, nhidden, nblocks, itrain))
fsumm = os.path.join(Train.data_dir(), 'anpe_thetaunt_magsigz.%s.%ix%i.%i.p' % (sample, nhidden, nblocks, itrain))

anpe = Inference.SNPE(
        prior=prior, 
        density_estimator=Ut.posterior_nn('maf', hidden_features=nhidden, num_transforms=nblocks), 
        device=device)
anpe.append_simulations(
    torch.as_tensor(x_train.astype(np.float32)), 
    torch.as_tensor(y_train.astype(np.float32)))
p_x_y_estimator = anpe.train()#dataloader_kwargs={'num_workers': 4})

KeyboardInterrupt: 

In [10]:
anpe._summary.keys()

dict_keys(['epochs_trained', 'best_validation_log_prob', 'validation_log_probs', 'training_log_probs', 'epoch_durations_sec'])

In [11]:
# save trained ANPE
torch.save(p_x_y_estimator.state_dict(), fanpe)

# save training summary
print(anpe._summary['best_validation_log_prob'])
pickle.dump(anpe._summary, open(fsumm, 'wb'))

[0.2575466987831406]
